In [1]:
import numpy as np
import os, sys
import importlib
from tqdm.notebook import tqdm, trange

import torch
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

from SimilarityVLM import SimilarityVLM
from dataset.dataset import FewShotTaskDataset
from FewShotClassifier import FewShotClassifier

### Choose VLM to Test

Note, this notebook must be run using the corresponding conda environment

In [2]:
VLM = importlib.import_module("VT-TWINS.wrapper").VTTWINS_SimilarityVLM

### Choose Dataset to Test

In [3]:
DATASET_SPLIT_PATH = "D:/datasets/PAC/few_shot_act_reg/kinetics_100_split/test.txt"

### Testing Function

In [4]:
def few_shot_accuracy(classifier: FewShotClassifier, dataset_split_path: str, n_way: int, n_support: int, n_query: int = 1, n_episodes: int = 1000) -> float:
    
    # Load dataset to generate tasks with the desired params
    dataset = FewShotTaskDataset(dataset_split_path, n_episodes, n_way, n_support, n_query)
    
    correct_predictions = 0
    total_queries = 0
    for vid_paths, category_names in tqdm(dataset):
        
        query_vid_paths = vid_paths[:, n_support:]
        if n_support > 0:
            support_vid_paths = vid_paths[:, :n_support]
        else:
            support_vid_paths = None
            
        query_predictions = classifier.predict(category_names, support_vid_paths, query_vid_paths)
        
        correct_predictions += np.sum(query_predictions == np.arange(n_way)[:, None])
        total_queries += n_way * n_query
        
    return correct_predictions / total_queries

### Run the Test

In [5]:
vlm = VLM()
classifier = FewShotClassifier(vlm, metric=None)

In [6]:
few_shot_accuracy(classifier, DATASET_SPLIT_PATH, 5, 5, 2, 20)

  0%|          | 0/20 [00:00<?, ?it/s]

d:\OneDrive\code\Stanford\PAC\vlm_benchmark\VT-TWINS\VT-TWINS\loader\msrvtt_loader.py:88: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_numpy.cpp:178.)
  video = th.from_numpy(video)
